# JST

In [4]:
import numpy as np

import pandas as pd
from pandarallel import pandarallel
from reviews.config import processed_data_dir, asum_input_dir, data_dir
from reviews.preprocess import preprocess

pandarallel.initialize(progress_bar=True, nb_workers=8)


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
reviews_df = pd.read_json(
    processed_data_dir / "products_reviews.json.gz", orient="records"
)
reviews_df = reviews_df[:1000]

In [6]:
tokens = reviews_df["text"].apply(lambda x: preprocess(x, lemmatization=True, sentences=False, return_tokens=False))

In [65]:
docs = list(tokens)
docs = [f"d{i} {x}" for i, x in enumerate(docs)]

docs_file = "\n".join(docs)

with open(data_dir / "jst" / "docs.dat", "w") as f:
    f.write(docs_file)

In [26]:
vocabs = [x for sublist in list(tokens) for x in sublist.split(" ")]

In [34]:
from collections import Counter
vocab_file = "\n".join([f"{k} {v}"  for (k, v) in dict(Counter(vocabs)).items()])

with open(data_dir / "jst" / "wordmap.txt", "w") as f:
    f.write(vocab_file)

In [ ]:
from reviews.config import data_dir, bin_dir, out_dir
from reviews.models import JST

alpha = 0.1
beta = 0.001 # [0.001, 0.1, 0]
gamma = 1 #[1, 1]
n_topics = 20
iterations = 1000

model = JST(bin_dir, data_dir / "jst", out_dir / "jst")
model.estimate(alpha, beta, gamma, n_topics, iterations)